In [39]:
import cv2
import numpy as np
import pandas as pd
import random
import matplotlib.colors as mcolors
import sys
from IPython.display import Image, display

In [40]:
aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [41]:
cap = cv2.VideoCapture("src/mov/test02.mp4")

if not (cap.isOpened()) :   # 正常に読み込めなかった場合終了する（VideoCaptureコンストラクタ自体は失敗してもFalseを返さないので注意）
    sys.exit()
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("FPS: " + str(fps))
frameCount = 1
ret, frame = cap.read()
h, w = frame.shape[:2]

def imshow(img, window_name="img"):
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

FPS: 29


In [42]:
# 実験対象のフレームを画像に保存
frame_want = np.arange(199, 209)
for frame in frame_want:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret, img = cap.read()
    cv2.imwrite("./output/noisy_frames/test02_" + str(frame) + ".jpg", img)

In [51]:
img = cv2.imread("./output/noisy_frames/test02_199.jpg")
# ガウシアンフィルタ
img_ga = cv2.GaussianBlur(img, (9, 9), 1)   # フィルターサイズは必ず奇数を与える
img_bi = cv2.bilateralFilter(img_ga, 20, 20, 30, 30)
imshow(img_bi)

In [54]:
def make_sharp_kernel(k: int):
  return np.array([
    [-k / 9, -k / 9, -k / 9],
    [-k / 9, 1 + 8 * k / 9, k / 9],
    [-k / 9, -k / 9, -k / 9]
  ], np.float32)

kernel = make_sharp_kernel(3)
img_ga = cv2.GaussianBlur(img, (9, 9), 1)   # フィルターサイズは必ず奇数を与える
img_sp = cv2.filter2D(img_ga, -1, kernel).astype("uint8")
imshow(img_sp)